In [15]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as scp
import pingouin as pg
import os

sns.set(context='talk',style='white')

hbn_folder = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study'
project_folder = '/Users/catcamacho/Library/CloudStorage/Box-Box/SEAL/stickystates'
ts_folder = os.path.join(hbn_folder, 'proc','group','parcel_timeseries','sub_ts')
data_folder = os.path.join(project_folder, 'DATA','hbn')
out_folder = os.path.join(project_folder, 'ANALYSIS','hbn_brain_states_final', 'k3')

sample_file = os.path.join(hbn_folder,'social_proc_networks','dynamic_connectivity','DATA', 
                           'helper_files','sample_gord.32k_fs_LR.pscalar.nii')
# get parcel and network labels
parcel_labels = nib.load(sample_file).header.get_axis(1).name
network_labels = []
for s in parcel_labels:
    b = s.split('_')
    if len(b)<2:
        network_labels.append(b[0])
    else:
        network_labels.append(b[1])
network_labels = np.array(network_labels)
network_names, network_sizes = np.unique(network_labels, return_counts=True)

# define measures of interest
networks_of_interest = ['Auditory', 'CinguloOperc', 'Default', 'DorsalAttn', 'FrontoParietal',
                        'SMhand', 'SMmouth', 'Salience', 'VentralAttn', 'Visual']
networks_palette = ['#FF00FF','#800080','#FF0000','#00FF00','#FFFF00','#00FFFF','#FF8000',
                    '#000000','#008080','#0000BD']

features_of_interest = ['Positive','Negative','Anger','Happy','Fear','Sad','Excited','Brightness',
                        'SaliencyFract','Sharpness','Vibrance','Loudness','Motion']
timing = np.round(np.arange(0,600,0.8),1)
movie='DM'

n_states = 3

state_palette = ['#05159b', '#cf28cf', '#107070']

# Compute SCARED alpha reliability

In [14]:
# load sample data
subinfo = pd.read_csv(os.path.join(data_folder, 'sampleinfo_movie{0}.csv'.format(movie)), index_col=0)

# load anxiety data
item_info = {'SR_SC':['SCARED_SR_3','SCARED_SR_10','SCARED_SR_26','SCARED_SR_32',
                      'SCARED_SR_39','SCARED_SR_40','SCARED_SR_41'], 
             'PR_SC':['SCARED_P_03','SCARED_P_10','SCARED_P_26','SCARED_P_32',
                      'SCARED_P_39','SCARED_P_40','SCARED_P_41'], 
             'PR_GD':['SCARED_P_05','SCARED_P_07','SCARED_P_14','SCARED_P_21',
                      'SCARED_P_23','SCARED_P_28','SCARED_P_33','SCARED_P_35','SCARED_P_37'], 
             'SR_GD':['SCARED_SR_5','SCARED_SR_7','SCARED_SR_14','SCARED_SR_21',
                      'SCARED_SR_23','SCARED_SR_28','SCARED_SR_33','SCARED_SR_35','SCARED_SR_37'], 
             'all_SR':['SCARED_SR_{0}'.format(a) for a in range(1,42)], 
             'all_PR':['SCARED_P_{0}'.format(str(a).zfill(2)) for a in range(1,42)]}

scared_sr = pd.read_csv(os.path.join(hbn_folder, 'phenotypic_data','9994_SCARED_SR_20210322.csv'), 
                        header=0, skiprows=[1], index_col='EID').loc[:,item_info['all_SR']]
scared_sr.index = ['sub-{0}'.format(i) for i in scared_sr.index]

scared_pr = pd.read_csv(os.path.join(hbn_folder, 'phenotypic_data','9994_SCARED_P_20210322.csv'), 
                        header=0, skiprows=[1], index_col='EID').loc[:,item_info['all_PR']]
scared_pr.index = ['sub-{0}'.format(i) for i in scared_pr.index]

#merge datasets together
full_data = subinfo.merge(scared_pr, how='left', left_index=True, right_index=True)
full_data = full_data.merge(scared_sr, how='left', left_index=True, right_index=True)
full_data = full_data.drop_duplicates()
full_data.describe()

,age,female,meanFD,righthanded,PPS_score,sample,SCARED_P_01,SCARED_P_02,SCARED_P_03,SCARED_P_04,...,SCARED_SR_32,SCARED_SR_33,SCARED_SR_34,SCARED_SR_35,SCARED_SR_36,SCARED_SR_37,SCARED_SR_38,SCARED_SR_39,SCARED_SR_40,SCARED_SR_41
count,620.000000,620.000000,620.000000,620.000000,439.000000,620.000000,545.000000,543.000000,545.000000,545.000000,...,431.000000,432.000000,432.000000,432.000000,432.000000,431.000000,431.000000,432.000000,432.000000,432.000000
mean,10.433548,0.404839,0.359507,0.750000,9.739749,1.500000,0.161468,0.314917,0.686239,0.337615,...,0.839907,0.773148,0.280093,0.657407,0.261574,0.540603,0.259861,0.666667,0.664352,0.668981
std,2.807872,0.491257,0.184016,0.433362,4.254149,0.500404,0.432568,0.551999,0.685119,0.593901,...,0.798186,0.761255,0.599901,0.742005,0.568798,0.693893,0.564236,0.761608,0.762112,0.736305
min,5.061259,0.000000,0.054385,0.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.165183,0.000000,0.209152,0.750000,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.164328,0.000000,0.315161,1.000000,8.000000,1.500000,0.000000,0.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,12.845824,1.000000,0.485037,1.000000,13.000000,2.000000,0.000000,1.000000,1.000000,1.000000,...,1.500000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,15.974332,1.000000,0.908530,1.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [19]:
 for k in item_info.keys():
     a = pg.cronbach_alpha(full_data.loc[:, item_info[k]])
     print('Cronbach Alpha for {0}: {1}'.format(k, np.round(a[0],2)))

Cronbach Alpha for SR_SC: 0.86
Cronbach Alpha for PR_SC: 0.88
Cronbach Alpha for PR_GD: 0.86
Cronbach Alpha for SR_GD: 0.86
Cronbach Alpha for all_SR: 0.94
Cronbach Alpha for all_PR: 0.91


# Examine differences by diagnosis